<a href="https://colab.research.google.com/github/HemanthhVV/Apache-Beam/blob/main/Beam_NoteBook_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache-beam[gcp]

In [ ]:
!pip install apache-beam

In [ ]:
!pip install apache-beam[interactive]

Understanding the concepts of Pipelines,Pcollections,Ptransforms

In [4]:
#Basic Imports for the Further Processing
import argparse
import logging
import re

import apache_beam as beam
from apache_beam.io import ReadFromText,WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

In [8]:
#Creating the class for the each transformation for the data
class Output(beam.PTransform):
    class _Transformation1(beam.DoFn):#Simply, DoFn is used for the creating Transformations
        def process(self,element):
            yield element," Transformation 1 Done 👌"
    class _Transformation2(beam.DoFn):
        def process(self,element):
            yield element," Transformation 2 Done "
    class _PrintingAfterTransformation(beam.DoFn):
        def process(self,element):
            print(list(','.join(tple) for tple in element) ," This is the final elemnt after the two transformations")
    def expand(self,input):#Arranging the transformations orders for our PTransform class
          #Pardo is used for applying the Transformataion which is created above
        input | beam.ParDo(self._Transformation1()) \
        | beam.ParDo(self._Transformation2()) \
        | beam.ParDo(self._PrintingAfterTransformation())


In [9]:
with beam.Pipeline() as p:
    (p | beam.Create(["Sample Test text"]) | Output()) #Calling the Output Transformation class

['Sample Test text, Transformation 1 Done 👌', ' ,T,r,a,n,s,f,o,r,m,a,t,i,o,n, ,2, ,D,o,n,e, ']  This is the final elemnt after the two transformations


In [10]:
%%html
<style>
  /* Define your custom styles here */
  .custom-cell {
    background-color: #f25022;
    border: 1px solid #ccc;
    border-radius: 5px;
    padding: 10px;
    margin-bottom: 10px;
    box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
  }
</style>

<div class="custom-cell">
  <h3>This is a styled cell</h3>
  <p>You can add your content here.</p>
</div>


In [11]:
with beam.Pipeline() as p:#Creating the inline Pcollections
    p | beam.Create([
        "Thus , HOw was the class?",
        "",
        "Enf of line",
        "That's it"
    ]) | beam.Filter(lambda line : line!="") | beam.Map(print)


Thus , HOw was the class?
Enf of line
That's it


In [29]:
class Output(beam.PTransform):
    class _Transform(beam.DoFn):
        def __init__(self,prefix):
            super().__init__
            self.prefix = prefix
        def process(self,elemnt):
            print(self.prefix + str(elemnt))
    def __init__(self,prefix = ''):
        super().__init__
        self.prefix = prefix
    def expand(self,input):
        input | beam.ParDo(self._Transform(self.prefix))


In [32]:
with beam.Pipeline() as p:
    p | beam.io.ReadFromText('/content/kinglear.txt') \
    | beam.FlatMap(lambda sentence:sentence.split()) \
    | beam.Filter(lambda word: not word.isspace() or word.isalnum()) \
    | beam.combiners.Sample.FixedSizeGlobally(10) \
    | beam.FlatMap(lambda word: word) \
    | Output(prefix = 'Words->')


Words->turn
Words->All
Words->cause;
Words->KING
Words->a
Words->I
Words->in
Words->this
Words->pilgrimage:
Words->my


In [43]:
import pandas as pd
df = pd.read_csv('/content/sample1000.csv')
df.columns[16]
df["VendorID"].value_counts()

1    574
2    424
Name: VendorID, dtype: int64

In [44]:
df.isna().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64

In [45]:
class Output(beam.PTransform):
    class _OutputFn(beam.DoFn):
        def __init__(self, prefix=''):
            super().__init__()
            self.prefix = prefix

        def process(self, element):
            print(self.prefix+str(element))

    def __init__(self, label=None,prefix=''):
        super().__init__(label)
        self.prefix = prefix

    def expand(self, input):
        input | beam.ParDo(self._OutputFn(self.prefix))

Just checking any missing values in the row(line) , if Not then, We are taking that line(row)'s total_amount else, ignored it


In [48]:
class ExtractTaxiRideCostFn(beam.DoFn):

    def process(self, element):
        line = element.split(',')
        return tryParseTaxiRideCost(line,16)


def tryParseTaxiRideCost(line,index):
    if(len(line) > index):
        # print(line[index])
        yield line[index]
    else:
        # print(line)
        yield 0.0

In [49]:
with beam.Pipeline() as p:
  lines = p | 'Log lines' >> beam.io.ReadFromText('sample1000.csv') \
            | beam.ParDo(ExtractTaxiRideCostFn()) \
            | beam.combiners.Sample.FixedSizeGlobally(10) \
            | beam.FlatMap(lambda cost: cost) \
            | Output(prefix = 'Taxi cost: ')


Taxi cost: 8.3
Taxi cost: 7.54
Taxi cost: 12.3
Taxi cost: 24.95
Taxi cost: 19.1
Taxi cost: 44.8
Taxi cost: 16.8
Taxi cost: 32.8
Taxi cost: 23.8
Taxi cost: 27.8


# ***Beam Concepts***
**Pipeline** - A pipeline is a user-constructed graph of transformations that defines the desired data processing operations.


---


**PCollection** - A PCollection is a data set or data stream. The data that a pipeline processes is part of a PCollection.



---


**PTransform** - A PTransform (or transform) represents a data processing operation, or a step, in your pipeline. A transform is applied to zero or more PCollection objects, and produces zero or more PCollection objects.

---


**Aggregation** - Aggregation is computing a value from multiple (1 or more) input elements.User-defined function (UDF) - Some Beam operations allow you to run user-defined code as a way to configure the transform.

---

**Schema** - A schema is a language-independent type definition for a PCollection. The schema for a PCollection defines elements of that PCollection as an ordered list of named fields.

---

**SDK** - A language-specific library that lets pipeline authors build transforms, construct their pipelines, and submit them to a runner.

---

**Runner** - A runner runs a Beam pipeline using the capabilities of your chosen data processing engine.

---

**Window** - A PCollection can be subdivided into windows based on the timestamps of the individual elements. Windows enable grouping operations over collections that grow over time by dividing the collection into windows of finite collections.

---

**Watermark** - A watermark is a guess as to when all data in a certain window is expected to have arrived. This is needed because data isn’t always guaranteed to arrive in a pipeline in event time order, or to always arrive at predictable intervals.

---

**Trigger** - A trigger determines when to aggregate the results of each window.State and timers - Per-key state and timer callbacks are lower level primitives that give you full control over aggregating input collections that grow over time.

---

**Splittable DoFn** - Splittable DoFns let you process elements in a non-monolithic way. You can checkpoint the processing of an element, and the runner can split the remaining work to yield additional parallelism.